In [21]:
%%timeit

import aocd

data = [ (v[0], int(v[1:])) for v in aocd.get_data(day=1, year=2025).splitlines() ]
part1, part2, cur, pd = 0, 0, 50, "R"

for d,v in data:
    if d != pd: pd, cur = d, (100-cur)%100
    md, cur = divmod(cur+v,100)
    part1, part2 = part1 + (cur == 0), part2 + md
    
#print("Part1:",part1,"Part2:",part2)

3.19 ms ± 648 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit

import aocd

data = [ v.split("-") for v in aocd.get_data(day=2, year=2025).split(",") ]
part1, part2 = 0, 0

def bad(fro, to, rep):
    min_l = (len(fro)-1)//rep+1
    max_l = len(to)//rep
    if min_l>max_l: return set()
    mi = (10**(min_l-1)) if len(fro)%rep else (int(fro[:min_l]) + (fro[:min_l]*(rep-1)<fro[min_l:]))
    ma = (10**(max_l)-1) if len(to)%rep else (int(to[:max_l]) - (to[:max_l]*(rep-1)>to[max_l:]))
    return {int(str(v)*rep) for v in range(mi,ma+1)}

for fro,to in data:
    s = set(bad(fro,to,2))
    part1 += sum(s)
    for k in range(3,len(to)+1): s.update(bad(fro,to,k))
    part2 += sum(s)

#print("Part1:",part1,"Part2:",part2)

872 μs ± 139 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
%%timeit

import aocd

data = [ v.split("-") for v in aocd.get_data(day=2, year=2025).split(",") ]
part1, part2 = 0, 0

# 2 3 5 -6 -10 

def bad(fro, to, rep):
    ret = 0
    min_l = (len(fro)-1)//rep+1
    max_l = len(to)//rep
    if min_l>max_l: return ret
    mi = (10**(min_l-1)) if len(fro)%rep else (int(fro[:min_l]) + (fro[:min_l]*(rep-1)<fro[min_l:]))
    ma = (10**(max_l)-1) if len(to)%rep else (int(to[:max_l]) - (to[:max_l]*(rep-1)>to[max_l:]))
    mil = len(str(mi))
    mal = len(str(ma))
    for l in range(mil,mal+1):
        if l == mil: tmi = mi
        else: tmi = 10*(mil-1)
        if l == mal: tma = ma
        else: tma = 10*(mil) - 1
        ret += (tmi+tma)*(tma-tmi+1)//2*sum((10**(k*l)) for k in range(rep))
    return ret

for fro,to in data:
    s = bad(fro,to,2)
    part1 += s
    part2 += s
    for k in [3,5,7,-6,-10]: part2 += k//abs(k)*bad(fro,to,abs(k))

#print("Part1:",part1,"Part2:",part2)



634 μs ± 54.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [25]:
%%timeit

import aocd

def getjolt(batt,k):
    res, ret, cur, n = batt[-k:], 0, -1, len(batt)
    for i in range(len(batt)):
        while cur>=0 and res[cur]<batt[i] and n-i>=k-cur: cur -= 1
        if cur<k-1: res[cur+1], cur = batt[i], cur+1
    for v in res: ret = ret*10+v
    return ret

input = aocd.get_data(day=3, year=2025)
data = [[int(char) for char in line] for line in input.splitlines()]

#print
("Part1:",sum(map(lambda x:getjolt(x,2),data)),"Part1:",sum(map(lambda x:getjolt(x,12),data)))

8.39 ms ± 352 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%%timeit

import aocd
import numpy as np
input = aocd.get_data(day=4, year=2025)

mt = np.matrix([[c=="@" for c in line] for line in input.splitlines()])
h, w = mt.shape

def removable_rolls(mt):
    t = np.zeros((h,w))
    t[:h-1,:w-1] += mt[1:,1:]
    t[1:,1:] += mt[:h-1,:w-1]

    t[:h-1,:] += mt[1:,:]
    t[1:,:] += mt[:h-1,:]

    t[:,1:] += mt[:,:w-1]
    t[:,:w-1] += mt[:,1:]

    t[:h-1,1:] += mt[1:,:w-1]
    t[1:,:w-1] += mt[:h-1,1:]
    return np.logical_and((t<4),mt)

rm = removable_rolls(mt)
rm_size = np.matrix.sum(rm)
part1 = part2 = rm_size
while rm_size:
    mt = np.logical_xor(rm, mt)
    rm = removable_rolls(mt)
    rm_size = np.matrix.sum(rm)
    part2 += rm_size

#print("Part1:",part1,"Part2:",part2)



20.5 ms ± 1.96 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
%%timeit

import aocd
import numpy as np
input = aocd.get_data(day=4, year=2025)

mat = {(i,j) for i,line in enumerate(input.splitlines()) for j,c in enumerate(line) if c == "@" }

def neb(i,j): return({(i-1,j-1),(i-1,j+1),(i-1,j),(i,j-1),(i,j+1),(i+1,j-1),(i+1,j),(i+1,j+1)})

acc = { p for p in mat if len(neb(*p) & mat)<4 }
part1 = part2 = len(acc)
while len(acc):
    mat -= acc
    acc = { p for p in mat if len(neb(*p) & mat)<4 }
    part2 += len(acc)
    
#print("Part1:",part1,"Part2:",part2)


512 ms ± 26.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit

import aocd

input = aocd.get_data(day=5, year=2025)
import bisect

R,L = input.split("\n\n")
R = sorted((tuple(map(int,r.split("-"))) for r in R.splitlines()))
L = list(map(int,L.splitlines()))

R2 = []
cur = -1
for a,b in R:
    if a <= cur:
        if b>=cur:
            R2[-1] = (R2[-1][0],b)
            cur = b+1
    elif a > cur:
        R2.append((a,b))
        cur = b+1
        
part1 = sum(v>=R2[p][0] for v in L for p in [bisect.bisect_left(R2,v,key=lambda x:x[1])] if p<len(R2))
#print("Part1:",part1,"Part2:",sum(b-a+1 for a,b in R2))


1.65 ms ± 175 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
%%timeit

import aocd
from functools import reduce

input = aocd.get_data(day=6, year=2025)

data = input.splitlines()
nums = [list(map(int,line.split())) for line in data[:-1]]
signs = data[-1].split()

part1 = sum(reduce((lambda x,y:x+y) if signs[i]=="+" else (lambda x,y:x*y),
                   (nums[j][i] for j in range(len(nums)))) for i in range(len(signs)))

part2 = 0
op = (lambda x,y:x+y)
bloc = 0
for j in range(len(data[0])):
    cur = 0
    for i in range(len(data)-1):
        c = data[i][j]
        if c !=" ": cur = cur*10+(ord(c)-48)

    if cur == 0:
        part2 += bloc
        bloc = 0
        op = (lambda x,y:x+y)

    else: bloc = op(bloc,cur)

    if data[-1][j]!=" ":
        op = (lambda x,y:x+y) if data[-1][j]=="+" else (lambda x,y:x*y)
part2 += bloc

#print("Part1:",part1,"Part2:",part2)





6.56 ms ± 666 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%%timeit

import aocd
from functools import reduce

input = aocd.get_data(day=6, year=2025)

data = input.splitlines()
nums = [list(map(int,line.split())) for line in data[:-1]]
signs = data[-1].split()

part1 = sum(reduce((lambda x,y:x+y) if signs[i]=="+" else (lambda x,y:x*y),
                   (nums[j][i] for j in range(len(nums)))) for i in range(len(signs)))
                   
data2 = ["".join(data[i][j] for i in range(len(data)-1)).split() for j in range(len(data[0])) ]

part2 = 0
op = (lambda x,y:x+y)
bloc = 0
for i,l in enumerate(data2):
    if len(l):
        bloc = op(int(l[0]),bloc)
        if data[-1][i] != " ": op = (lambda x,y:x+y) if data[-1][i]=="+" else (lambda x,y:x*y)
    else: op, bloc, part2 = (lambda x,y:x+y), 0, part2+bloc
part2 += bloc

#print("Part1:",part1,"Part2:",part2)




10.8 ms ± 618 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
%%timeit
import aocd
from collections import Counter

input = aocd.get_data(day=7, year=2025)
data = input.splitlines()
beams = {i:1 for i,v in enumerate(data[0]) if v == "S"}
part1 = 0

for line in data[2::2]:
    nbeams = Counter()
    for i,c in beams.items():
        if line[i] == "^":
            part1 += 1
            nbeams[i+1] += c
            nbeams[i-1] += c
        else: nbeams[i] += c
    beams = nbeams
part2 = sum(beams.values())

#print("Part1:",part1,"Part2:",part2)

3.49 ms ± 538 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
